In [1]:
# add autoreload
%load_ext autoreload
%autoreload 2
import pandas as pd
import pyreadstat
import os

In [2]:
os.chdir('T:/laupodteam/AIOS/Bram/data')

In [3]:
argus = pd.read_csv('./Argus/joined_argus/celldyn_raw.csv', sep=';')[['fcs_id', 'afname_dt']]
argus = argus.assign(dataset='ARGUS')
argus = argus.drop_duplicates()

ucorbio = pd.read_sas('./Argus/external_validation/ucorbio/st2_celldyn_13sep2021.sas7bdat')[['fcs_id', 'afname_dt']]
ucorbio = ucorbio.assign(dataset='UCORBIO')
ucorbio = ucorbio.assign(fcs_id=ucorbio.fcs_id.str.decode('utf-8'))
ucorbio = ucorbio.drop_duplicates()

heartscore = pd.read_csv('./HeartScore/Data/full_celldyn.csv', sep=';')[['fcs_id', 'afname_dt']]
heartscore = heartscore.assign(dataset='HEARTSCORE')
heartscore = heartscore.drop_duplicates()

MDS = pd.read_sas('./MDS/celldyn_sapph_20220905.sas7bdat')[['fcs_id', 'afname_dt']]
MDS = MDS.assign(dataset='MDS')
MDS = MDS.assign(fcs_id=MDS.fcs_id.str.decode('utf-8'))
MDS = MDS.drop_duplicates()

Reuma, _ = pyreadstat.read_sas7bdat('./Reuma/DT2RA/celldyn.sas7bdat')
Reuma = Reuma[['fcs_id', 'afname_dt']]
Reuma = Reuma.assign(dataset='REUMA')
Reuma = Reuma.drop_duplicates()

MDL, _ = pyreadstat.read_sas7bdat('./MDL/sapphire_19aug2021.sas7bdat')
MDL = MDL[['fcs_id', 'afname_dt']]
MDL = MDL.assign(dataset='MDL')
MDL = MDL.drop_duplicates()

LABELS = pd.concat([argus, ucorbio, heartscore, MDS, Reuma, MDL], axis=0)
LABELS = LABELS.assign(fcs_id=LABELS.fcs_id.str.strip())
LABELS = LABELS[LABELS.fcs_id.str.strip()!='']

C:\Users\bes3\AppData\Local\Temp/ipykernel_22320/2824107347.py:1: DtypeWarning: Columns (3,4,17,210) have mixed types. Specify dtype option on import or set low_memory=False.
  argus = pd.read_csv('./Argus/joined_argus/celldyn_raw.csv', sep=';')[['fcs_id', 'afname_dt']]


In [5]:
celldyn = pd.read_feather("./CellDyn/celldyn.feather")
celldyn = celldyn[['studyId_Alle_celldyn', 'fcs_id', 'afname_dt']]
celldyn = celldyn.assign(fcs_id=celldyn.fcs_id.str.decode('utf-8'))
celldyn = celldyn.set_index('fcs_id')

In [ ]:
cnts = LABELS.groupby('fcs_id').size()
mults = cnts[cnts>1].index.tolist()

In [ ]:
LABELS = LABELS.groupby('fcs_id').dataset.agg(lambda x: "_".join(set(x))).to_frame()

In [ ]:
celldyn.join(LABELS, how='left').reset_index().to_parquet('./CellDyn/artifacts/celldyn_labels.parquet')